In [1]:
import pandas as pd; pd.set_option('display.max_rows', None)
from bs4 import BeautifulSoup as BS
import requests

BASE_URL = "https://www.fantasypros.com/nfl/adp/overall.php"

def make_adp_df():
    res = requests.get(BASE_URL)
    if res.ok:
        soup = BS(res.content, 'html.parser')
        table = soup.find('table', {'id': 'data'})
        df = pd.read_html(str(table))[0]
        print('Output after reading the html:\n\n', df.head(), '\n') # so you can see the output at this point
        df = df[['Player Team (Bye)', 'POS', 'AVG']]
        print('Output after filtering:\n\n', df.head(), '\n')
        df['PLAYER'] = df['Player Team (Bye)'].apply(lambda x: ' '.join(x.split()[:-2])) # removing the team and position
        df['POS'] = df['POS'].apply(lambda x: x[:2]) # removing the position rank
        
        df = df[['PLAYER', 'POS', 'AVG']].sort_values(by='AVG')
        
        print('Final output: \n\n', df.head())
        
        return df
        
    else:
        print('oops, something didn\'t work right', res.status_code)
        
df = make_adp_df()

Output after reading the html:

    Rank             Player Team (Bye)  POS  MFL  Fantrax  FFC  AVG
0     1  Christian McCaffrey CAR (13)  RB1  1.0      1.0  1.0  1.0
1     2           Dalvin Cook MIN (7)  RB2  2.0      2.0  3.0  2.3
2     3        Derrick Henry TEN (13)  RB3  3.0      3.0  2.0  2.7
3     4           Alvin Kamara NO (6)  RB4  4.0      4.0  4.0  4.0
4     5       Saquon Barkley NYG (10)  RB5  5.0      5.0  9.0  6.3 

Output after filtering:

               Player Team (Bye)  POS  AVG
0  Christian McCaffrey CAR (13)  RB1  1.0
1           Dalvin Cook MIN (7)  RB2  2.3
2        Derrick Henry TEN (13)  RB3  2.7
3           Alvin Kamara NO (6)  RB4  4.0
4       Saquon Barkley NYG (10)  RB5  6.3 

Final output: 

                 PLAYER POS  AVG
0  Christian McCaffrey  RB  1.0
1          Dalvin Cook  RB  2.3
2        Derrick Henry  RB  2.7
3         Alvin Kamara  RB  4.0
4       Saquon Barkley  RB  6.3


In [2]:
replacement_players = {
    'RB': '',
    'WR': '',
    'TE': '',
    'QB': ''
}

for _, row in df[:100].iterrows():
    position = row['POS']
    player = row['PLAYER']
    replacement_players[position] = player


In [3]:
replacement_players

{'RB': 'Zack Moss',
 'WR': 'Will Fuller V',
 'TE': 'Logan Thomas',
 'QB': 'Joe Burrow',
 'DS': 'Pittsburgh Steelers'}

In [4]:
# each position has a different associated URL. We'll create a string format here and loop through the possible positions
BASE_URL = 'https://www.fantasypros.com/nfl/projections/{position}.php?week=draft'

def make_projection_df():
    
    # we are going to concatenate our individual position dfs into this larger final_df
    final_df = pd.DataFrame()
    
    #url has positions in lower case
    for position in ['rb', 'qb', 'te', 'wr']:
        
        res = requests.get(BASE_URL.format(position=position)) # format our url with the position
        if res.ok:
            soup = BS(res.content, 'html.parser')
            table = soup.find('table', {'id': 'data'})
            df = pd.read_html(str(table))[0]
            
            df.columns = df.columns.droplevel(level=0) # our data has a multi-level column index. The first column level is useless so let's drop it.
            df['PLAYER'] = df['Player'].apply(lambda x: ' '.join(x.split()[:-1])) # fixing player name to not include team
            # if you're not doing PPR, don't include this. If you're doing Half PPR,
            # multiply receptions * 1/2
            if 'REC' in df.columns:
                df['FPTS'] = df['FPTS'] + df['REC'] # add receptions if they're in there. 
            
            df['POS'] = position.upper() # add a position column
            
            df = df[['PLAYER', 'POS', 'FPTS']]
            final_df = pd.concat([final_df, df]) # iteratively add to our final_df
        else:
            print('oops something didn\'t work right', res.status_code)
            return
    
    final_df = final_df.sort_values(by='FPTS', ascending=False) # sort df in descending order on FPTS column
    
    return final_df
            
df = make_projection_df()
df.head()

,PLAYER,POS,FPTS
0,Christian McCaffrey,RB,397.8
0,Josh Allen,QB,378.6
1,Patrick Mahomes II,QB,376.4
2,Kyler Murray,QB,357.7
3,Lamar Jackson,QB,349.2


In [5]:
replacement_values = {
    'RB': 0,
    'WR': 0,
    'QB': 0,
    'TE': 0
}

for position, player in replacement_players.items():
    if position in ['QB', 'WR', 'TE', 'RB']:
        replacement_values[position] = df.loc[df['PLAYER'] == player].values[0, -1]
    
replacement_values

{'RB': 146.8, 'WR': 181.9, 'QB': 286.8, 'TE': 156.5}

In [6]:
df['VOR'] = df.apply(
    lambda row: row['FPTS'] - replacement_values.get(row['POS']), axis=1
)
df.head()


,PLAYER,POS,FPTS,VOR
0,Christian McCaffrey,RB,397.8,251.0
0,Josh Allen,QB,378.6,91.8
1,Patrick Mahomes II,QB,376.4,89.6
2,Kyler Murray,QB,357.7,70.9
3,Lamar Jackson,QB,349.2,62.4


In [8]:
df = df.sort_values(by='VOR', ascending=False)
df['VALUERANK'] = df['VOR'].rank(ascending=False)
df.head(200)
df.to_csv(r'C:\Users\bradl\Desktop\\2021_Fantasy_Football_Rankings.csv')